In [1]:
Pkg.update() 
Pkg.add("JuMP")

INFO: Updating METADATA...
INFO: Updating PGFPlots master...
INFO: Computing changes...
INFO: No packages to install, update or remove


In [2]:
using JuMP

INFO: Package JuMP is already installed


In [17]:
include("../utils/util.jl")
include("../utils/network.jl")
include("../utils/problem.jl")
include("maxSens.jl")

gridPartition (generic function with 1 method)

In [4]:
small_nnet = read_nnet("../reverify/small_nnet.txt")

Network(Layer[Layer([1.0; 1.0], [1.5, 1.5]), Layer([2.0 2.0; 2.0 2.0], [2.5, 2.5]), Layer([3.0 3.0], [3.5])])

In [5]:
function MaxSens(nnet::Network,x0::Vector{Float64},delta0::Float64,actFunc::Function)
    n_layer = length(nnet.layers)
    x = Vector{Vector{Float64}}(n_layer+1)
    x[1] = x0
    delta = Vector{Float64}(n_layer+1)
    delta[1] = delta0
    for i in 1:n_layer
        n_node = size(nnet.layers[i].weights)[1]
        gamma = Vector{Float64}(n_node)
        input = nnet.layers[i].weights * x[i] + nnet.layers[i].bias
        for j in 1:n_node
            beta = getBounds(x[i],delta[i],nnet.layers[i].weights[j,:],nnet.layers[i].bias[j])
            gamma[j] = getGamma(actFunc, input[j], beta)
        end
        x[i+1] = actFunc(input)
        delta[i+1] = maximum(gamma)
    end
    return [x[n_layer+1],delta[n_layer+1]]
end

ff(x) = ReLU(x)
x, theta = MaxSens(small_nnet, [1.0], .3, ff)

2-element Array{Any,1}:
  [78.5]
 7.2    

In [30]:
function OutputReach(nnet::Network, inputSet::Constraints, delta::Float64, actFunc::Function)
    x = gridPartition(inputSet, delta)
    n_grid = length(x)
    y = Vector{Vector{Float64}}(n_grid)
    epsilon = Vector{Float64}(n_grid)
    for i in 1:n_grid
        output = MaxSens(nnet, x[i], delta, ff)
        y[i] = output[1]
        epsilon[i] = output[2]
    end
    return [y, epsilon]
end

inputSet = Constraints(Matrix(0,0),[],[1.0],[-1.0])
OutputReach(small_nnet, inputSet, 0.3, ff)

2-element Array{Array{Any,1},1}:
 Any[[34.1], [41.3], [48.5], [55.7], [62.9], [70.1], [77.3]]
 Any[7.2, 7.2, 7.2, 7.2, 7.2, 7.2, 7.2]                     